In [1]:
using DataFrames, CSV, JuMP, Gurobi, Plots, Random

In [2]:
const GRB_ENV = Gurobi.Env(output_flag=0);

In [41]:
function team_match()
    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "TimeLimit", 180)

    # PARAMETERS
    N = 20 # teams
    W = 38

    # VARIABLES
    @variable(model, x[i = 1:N, j = 1:N, w = 1:W], Bin) # 1 if team i plays team j on week w, 0 otherwise

    # OBJECTIVE FUNCTION
    @objective(model, Max, sum(x[i,j,w] for i in 1:N, j in 1:N, w in 1:W))

    # CONSTRAINTS
    # teams can never play against themselves
    @constraint(model, [i = 1:N, w = 1:W], x[i, i, w] == 0)

    # each team can only play at most one game per week
    @constraint(model, [i = 1:N, w = 1:W], sum(x[i, j, w] + x[j, i, w] for j in 1:N) <= 1)

    for i=1:N
        for j=1:N
            if i!=j
                # each game is against a different team, team play against each other team at most once in first half of season
                @constraint(model, sum(x[i, j, w] + x[j, i, w] for w in 1:Int(W/2)) == 1)
                
                # each game is against a different team, team play against each other team at most once in second half of season
                @constraint(model, sum(x[i, j, w] + x[j, i, w] for w in Int((W/2))+1:W) == 1)

                # every team plays every other team once at home
                @constraint(model, sum(x[i, j, w] for w in 1:W) == 1)
            end
        end
    end

    # no more than 2 straight home games
    @constraint(model, [i = 1:N, w = 1:W-2], sum(x[i, :, w] + x[i, :, w+1] + x[i, :, w+2]) <= 2)

    # no more than 2 straight away games
    @constraint(model, [j = 1:N, w = 1:W-2], sum(x[:, j, w] + x[:, j, w+1] + x[:, j, w+2]) <= 2)


    # OPTIMIZE
    # solvetime = @elapsed optimize!(model)
    optimize!(model)


    return value.(x)
end

team_match (generic function with 1 method)

In [42]:
x = team_match()

20×20×38 Array{Float64, 3}:
[:, :, 1] =
  0.0  -0.0  -0.0  -0.0  -0.0   1.0  …   0.0  -0.0   0.0  -0.0   0.0  -0.0
 -0.0   0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0   0.0  -0.0  -0.0
  0.0  -0.0   0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0   0.0  -0.0   0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
  0.0  -0.0  -0.0  -0.0   0.0  -0.0     -0.0  -0.0   0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0   0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0   1.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0   0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0   1.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0   1.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0   1.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0      0.0  -0.0   0.0  -0

In [52]:
sum(x[:,:,1])

10.0

In [ ]:
function table_games_w(x, team)
    ## create w dataframes, one for each week, each dataframe contains the games for that week
    N = 20
    W = 38
    table = zeros(W,2)
    for w=1:W
        for j=1:N
            if x[team,j,w] == 1 && x[j,team,w] == 0
                table[w,1] = j
                table[w,2] = j 
            elseif x[team,j,w] == 0 && x[j,team,w] == 1
                table[w,1] = j
                table[w,2] = -j
            end
        end
    end
    return table
end

In [ ]:
# C is upper triangular matrix; each entry is the cost of playing in t and then t_
function GetC(x,y)
    return 1/(y-x)
end

In [ ]:
function schedule_days(x,u,z)
    N=20
    W=38
    D=4 #playable number of days

    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "TimeLimit", 180)

    # VARIABLES
    @variable(model, y[i = 1:N, w = 1:W, d = 1:D], Bin) # 1 if team i plays on day d of week w, 0 otherwise

    # OBJECTIVE FUNCTION
    @objective(model, Min, 
        sum(((1-u[i,w])*C(7 * w + d, 7 * (w+1) + d_) 
            + u[i,w]*(C(7 * w + d, 7 * (w+1) + z[i,w])+C(7 * (w+1) + z[i,w], 7 * (w+1) + d_)))
            *(x[i,j,w]+x[j,i,w])*(x[i,k,w+1]+x[k,i,w+1])*y[i,w,d]*y[i,w+1,d]
        for i in 1:N, j in 1:N, k in 1:N, w in Win:Wend-1, d in 1:D, d_ in 1:D))

    # CONSTRAINTS
    
    # teams who play each other on week w must play on the same day of the week
    @constraint(model, [i = 1:N, j = 1:N, w = 1:W, d = 1:D], y[i,w,d]*y[j,w,d] >= x[i,j,w] + x[j,i,w])

    # maximum of 1 game on day 1
    @constraint(model, [w = 1:W], sum(y[i,w,1] for i in 1:N) <= 2) #2 teams play per game

    # maximum of 4 games on day 2
    @constraint(model, [w = 1:W], sum(y[i,w,2] for i in 1:N) <= 8)

    # maximum of 4 games on day 3
    @constraint(model, [w = 1:W], sum(y[i,w,3] for i in 1:N) <= 8)

    # maximum of 1 game on day 4
    @constraint(model, [w = 1:W], sum(y[i,w,4] for i in 1:N) <= 2)


    # OPTIMIZE
    # solvetime = @elapsed optimize!(model)
    optimize!(model)


    return value.(y)
end

In [ ]:
y = schedule_days(x,u,z)